## This notebook reads the SENTINEL 1 COG from aws s3

This notebook deals with reading the cog file of sentinel 1 SAR data hosted on aws s3 bucket "Analysis Ready Sentinel-1 Backscatter Imagery". It reads the data in chunks, then plotting and visualizing it. For this proper libraries and their versions need to be installed. Since this notebook uses gdal, rasterio and a pythonic interface for aws s3 i.e., s3fs this takes proper system and formats to run it properly.
Further once the data is read properly by the url, then lots of analysis and processing can be done.
Here I have used jupyter notebook to code with python 3.11.4 . The time taken is also calculated for each action.


In [ ]:
#pip install s3fs

In [ ]:
#pip install xarray

In [ ]:
#pip install intake

In [ ]:
#pip install rio-cogeo==2.0a3

In [81]:
import dask
import s3fs
import intake
import os
import xarray 
import rasterio
import pandas as pd
import rioxarray as rxr

## Import gdal and set up environment

In [2]:
from osgeo import gdal

In [3]:
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           AWS_NO_SIGN_REQUEST='YES',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000')
os.environ.update(env)

## Fetched the data from aws s3.

In [66]:
s3 = s3fs.S3FileSystem(anon=True)
objects = s3.ls('sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/')
#https://raw.githubusercontent.com/scottyhq/sentinel1-rtc-stac/main/13SBD/2021/S1A_20210110_13SBD_DSC/S1A_20210110_13SBD_DSC.json
images = ['s3://' + obj + '/Gamma0_VH.tif' for obj in objects]
print(len(images))
images[:11] #january 2020 scenes

135


['s3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200110_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200117_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200122_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200129_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200203_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200210_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200215_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200227_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200310_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200317_14TPN_ASC/Gamma0_VH.tif',
 's3://sentinel-s1-rtc-indigo/

In [67]:
with open('files.txt', 'w') as f:
    lines = [x.replace('s3://', '/vsis3/') + '\n' for x in images[:6]]
    f.writelines(lines)

In [68]:
f

<_io.TextIOWrapper name='files.txt' mode='w' encoding='cp1252'>

## Builds a virtual file

In [69]:
%%time
!gdalbuildvrt stack.vrt -separate -input_file_list files.txt 

0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: total: 0 ns
Wall time: 9.98 s


In [70]:
%%time
chunks= dict(band=1, x=2745, y=2745)
da = rxr.open_rasterio('stack.vrt', chunks=chunks)  #rioxarray.open_rasterio
da

CPU times: total: 0 ns
Wall time: 18.1 ms


<xarray.DataArray (band: 6, y: 5490, x: 5490)>
dask.array<open_rasterio-36a14494959c535af6c0b26205285667<this-array>, shape=(6, 5490, 5490), dtype=float32, chunksize=(1, 2745, 2745), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int32 1 2 3 4 5 6
  * x            (x) float64 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05 7.098e+05
  * y            (y) float64 4.8e+06 4.8e+06 4.8e+06 ... 4.69e+06 4.69e+06
    spatial_ref  int32 0
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0

In [71]:
da = da.rename({'band':'time'})
da['time'] = [pd.to_datetime(x[60:68]) for x in images[:6]]

In [72]:
#pip install intake-xarray

## # Loading data into python objects

In [76]:
%%time
pattern = 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/{band}/Gamma0_VH.tif'
chunks=dict(band=1, x=2745, y=2745)
sources = intake.open_rasterio(images[:6], chunks=chunks, path_as_pattern=pattern, concat_dim='band')
da = sources.to_dask() 
da

CPU times: total: 46.9 ms
Wall time: 67.6 ms


<xarray.DataArray (band: 6, y: 5490, x: 5490)>
dask.array<concatenate, shape=(6, 5490, 5490), dtype=float32, chunksize=(1, 2745, 2745), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) <U22 'S1A_20200110_14TPN_ASC' ... 'S1A_20200210_14TPN...
  * x            (x) float64 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05 7.098e+05
  * y            (y) float64 4.8e+06 4.8e+06 4.8e+06 ... 4.69e+06 4.69e+06
    spatial_ref  int32 0
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     Gamma0_VH

## Dataclub from many COGs

In [77]:
%%time
chunks=dict(band=1, x=2745, y=2745)
dataArrays = [rxr.open_rasterio(url, chunks=chunks) for url in images]

# note use of join='override' b/c we know these COGS have the same coordinates
da = xarray.concat(dataArrays, dim='band', join='override', combine_attrs='drop')
da = da.rename({'band':'time'})
da['time'] = [pd.to_datetime(x[60:68]) for x in images]
da

CPU times: total: 1.23 s
Wall time: 1.44 s


<xarray.DataArray (time: 135, y: 5490, x: 5490)>
dask.array<concatenate, shape=(135, 5490, 5490), dtype=float32, chunksize=(1, 2745, 2745), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2020-01-10 2020-01-17 ... 2020-12-31
  * x            (x) float64 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05 7.098e+05
  * y            (y) float64 4.8e+06 4.8e+06 4.8e+06 ... 4.69e+06 4.69e+06
    spatial_ref  int32 0

## Plotting

 Using the bokeh library for this.

In [78]:
import hvplot.xarray
da.hvplot.image(rasterize=True, aspect='equal', cmap='gray', clim=(0,0.4))

:DynamicMap   [time]
   :Image   [y,x]   (value)